# Coroutines

The coroutines use the `yield` keyword as a way to stop and pass the execution to another routine. The coroutine is mostly like a generator with some differences. Check out [PEP342](https://www.python.org/dev/peps/pep-0342/) and PEP380.

__A simple coroutine__

In [33]:
def example():
    print('Starting')
    x = yield
    print('Continue coroutine + recieved value = ', x)

In [34]:
e = example()
e

<generator object example at 0x00C63370>

In [35]:
next(e)

Starting


In [36]:
e.send(10)

Continue coroutine + recieved value =  10


StopIteration: 

---

__Using inspect module to check the state of coroutine__

As you see above sometimes we use `next` and sometimes `send`, because before using `next` "the generator" had not started executing and `yield` was not waiting to receive any value using send.

In [37]:
import inspect

e = example()

inspect.getgeneratorstate(e)

'GEN_CREATED'

In [38]:
next(e)
inspect.getgeneratorstate(e)

Starting


'GEN_SUSPENDED'

In [39]:
e.send(10)

Continue coroutine + recieved value =  10


StopIteration: 

In [40]:
inspect.getgeneratorstate(e)

'GEN_CLOSED'

---

__Example 2__

An infinitly running co-routine averager which computes the average. The coroutine will terminate when `.close()` is called on it.

The advantage of using a coroutine is that sum and count can be simple local variables: no instance attributes or closures are needed to keep the context between calls.

In [13]:
def coroutine_averager():
    sum = 0
    count = 0
    while True:
        num = yield  # instead of print you can do "num = yield average" and skip the print statement at the end.
        sum += num
        count +=1 
        print(f"Average is {sum/count}")

In [14]:
a = coroutine_averager()

In [15]:
next(a)

In [16]:
a.send(10)

Average is 10.0


In [17]:
a.send(20)

Average is 15.0


---
__Priming by using decorator__

As you see its easy to forget doing `next(coroutine)` before `send` thus you can use a decorator to do that for you. Example shown below: - 

In [37]:
from functools import wraps

def coroutine(func):
    """
    This decorator primes the couroutine function by advancing it to the first yield
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return wrapper

In [38]:
@coroutine
def coroutine_averager():
    sum = 0
    count = 0
    while True:
        num = yield  # instead of print you can do "num = yield average" and skip the print statement at the end.
        sum += num
        count +=1 
        print(f"Average is {sum/count}")

In [39]:
a = coroutine_averager()

In [40]:
a.send(10)

Average is 10.0


In [41]:
a.send(15)

Average is 12.5


---
### Termination and Exception Handling

An unhandled exception within a coroutine propagates to the caller of the next or send that triggered it.

There are 2 ways to terminate a co-routine.

- Send something to the coroutine which it cannot cater to thus raising excpetion and terminating it. e.g. `None`
    - Also you can send `None` and when you check for `None` stop the while loop and return the value from the coroutine using return statement. Check example in _fluent python_
- Call the `.close()`

You can also send exceptions into the coroutine using `generator.throw()` - Causes the yield expression where the generator was paused to raise the exception given. Check further details in _Fluent Python_ page 472/473 generator section or [standard documentation](https://docs.python.org/3/reference/expressions.html#generator-iterator-methods).

Example below shows how you can handle exceptions in the coroutine. If you do handle the exception it will keep on running. With the `finally` you can do some teardown.

In [42]:
class SomeException(Exception):
    pass

def demo_coroutine():
    print("Coroutine started")
    try:
        while True:
            try:
                x = yield
            except SomeException:
                print("Exception recived but still continuing")
            else:
                print("Received x")  # or you can also put this in the try block
    finally:
        print("Coroutine ended")

---
__Returning values__

In [53]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return (count, average)

In [54]:
a = averager()
next(a)
a.send(10)

In [55]:
a.send(20)

In [56]:
a.send(None) # you do get the return but cannot stop the "StopIteration" exception.

StopIteration: (2, 15.0)

_Better way of handling StopIteration_  --> To be honest this seems like a hack to me as to get results from the generator and the book _fluent python_ agrees that this is somewhat of a hack. Much better to use `yeild from` in these scenarios because it tackles the StopIteration exception automatically which is transparent to the user.

In [57]:
# Better way

a = averager()
next(a)
a.send(10)
a.send(20)

In [59]:
try:
    a.send(None)
except StopIteration as e:
        result = e.value
finally:
    print(result)

(2, 15.0)


---
### Yield From

__Concept__

The main feature of `yield from` is to open a bidirectional channel from the outermost caller to the innermost subgenerator, so that values can be sent and yielded back and forth directly from them, and exceptions can be thrown all the way in without adding a lot of exception handling boilerplate code in the intermediate coroutines. This is what enables coroutine delegation in a way that was not possible before. 

The below should give you some idea about the construct...

    (caller)
    main func     <-->     delegating generator     <-->      subgenerator
    
While the delegating generator is suspended at `yield from` the main func can send data directly to subgenerator which yields data back to the main function. The delegating generator resumes when the subgenerator returns and the interpreter raises StopIteration with the returned value attached.

In [68]:
"""
Example from fluent python
"""

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# subgenerator
def averager():
    total = 0
    count = 0
    average = None
    while True:
        num = yield
        if num is None:  # to break the infinite for loop
            break
        total += num
        count += 1
        average = total / count
    return Result(count, average)


# delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()


# main function
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # important!
    # print(results) # uncomment to debug
    report(results)
    
    
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit))
        
        
        
data = {
'girls;kg':
[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
'girls;m':
[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
'boys;kg':
[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
'boys;m':
[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
